Step 0: A tiny, *runnable* bigram language model (no calculus), just pure NumPy.   
It learns "what letter tends to follow what letter" from a small text.   
Then we show sampling and a tiny self-attention demo with real numbers.  

In [2]:
import numpy as np
import pandas as pd


In [3]:
np.random.seed(0)

#### ---- Bigram Language model (count base) ----

In [5]:
text = (
    "tiny gpt built with pure numpy. "
    "this is a tiny demo to learn the math of transformers. "
    "it can overfit a short text."
)

In [6]:
text

'tiny gpt built with pure numpy. this is a tiny demo to learn the math of transformers. it can overfit a short text.'

#### 1) Build vocabulary (character-level)

In [13]:
# for get all character in text 
chars_set = set(text)
# convert it to list
chars_list = list(chars_set)
#sort it
chars = sorted(list(set(text)))

In [16]:
#mapping each text into number
stoi = {ch:i for i,ch in enumerate(chars)}
stoi
        

{' ': 0,
 '.': 1,
 'a': 2,
 'b': 3,
 'c': 4,
 'd': 5,
 'e': 6,
 'f': 7,
 'g': 8,
 'h': 9,
 'i': 10,
 'l': 11,
 'm': 12,
 'n': 13,
 'o': 14,
 'p': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'v': 20,
 'w': 21,
 'x': 22,
 'y': 23}

In [17]:
#swap between number that get from mapping and index in each dictionary in stoi
#สลับ ตำแหน่งกันระหว่างตัวอักษรกับ index
itos = {i:ch for ch,i in stoi.items()}
itos

{0: ' ',
 1: '.',
 2: 'a',
 3: 'b',
 4: 'c',
 5: 'd',
 6: 'e',
 7: 'f',
 8: 'g',
 9: 'h',
 10: 'i',
 11: 'l',
 12: 'm',
 13: 'n',
 14: 'o',
 15: 'p',
 16: 'r',
 17: 's',
 18: 't',
 19: 'u',
 20: 'v',
 21: 'w',
 22: 'x',
 23: 'y'}

In [21]:
V  = len (chars)
V

24

#### 2} Count bigrams (pairs of consecutive chars)

In [22]:
#create only mattrix size V x V that have only one
counts = np.ones((V, V), dtype = np.float64) # add-one smoothing to avoid zeros

In [23]:
for a, b in zip(text[:-1], text[1:]) : 
    counts[stoi[a], stoi[b] ] += 1 

In [ ]:
# tuple that consist of pair of cuple that have 1 letter and next letter togenther
tuple(zip(text[:-1], text[1:]))

(('t', 'i'),
 ('i', 'n'),
 ('n', 'y'),
 ('y', ' '),
 (' ', 'g'),
 ('g', 'p'),
 ('p', 't'),
 ('t', ' '),
 (' ', 'b'),
 ('b', 'u'),
 ('u', 'i'),
 ('i', 'l'),
 ('l', 't'),
 ('t', ' '),
 (' ', 'w'),
 ('w', 'i'),
 ('i', 't'),
 ('t', 'h'),
 ('h', ' '),
 (' ', 'p'),
 ('p', 'u'),
 ('u', 'r'),
 ('r', 'e'),
 ('e', ' '),
 (' ', 'n'),
 ('n', 'u'),
 ('u', 'm'),
 ('m', 'p'),
 ('p', 'y'),
 ('y', '.'),
 ('.', ' '),
 (' ', 't'),
 ('t', 'h'),
 ('h', 'i'),
 ('i', 's'),
 ('s', ' '),
 (' ', 'i'),
 ('i', 's'),
 ('s', ' '),
 (' ', 'a'),
 ('a', ' '),
 (' ', 't'),
 ('t', 'i'),
 ('i', 'n'),
 ('n', 'y'),
 ('y', ' '),
 (' ', 'd'),
 ('d', 'e'),
 ('e', 'm'),
 ('m', 'o'),
 ('o', ' '),
 (' ', 't'),
 ('t', 'o'),
 ('o', ' '),
 (' ', 'l'),
 ('l', 'e'),
 ('e', 'a'),
 ('a', 'r'),
 ('r', 'n'),
 ('n', ' '),
 (' ', 't'),
 ('t', 'h'),
 ('h', 'e'),
 ('e', ' '),
 (' ', 'm'),
 ('m', 'a'),
 ('a', 't'),
 ('t', 'h'),
 ('h', ' '),
 (' ', 'o'),
 ('o', 'f'),
 ('f', ' '),
 (' ', 't'),
 ('t', 'r'),
 ('r', 'a'),
 ('a', 'n'),
 ('n', 's'),

In [24]:
counts

array([[1., 1., 3., 2., 2., 2., 1., 1., 2., 1., 3., 2., 2., 2., 3., 2.,
        1., 2., 7., 1., 1., 2., 1., 1.],
       [3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [3., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 3., 1., 1.,
        2., 1., 2., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 2., 1., 1., 1., 1.],
       [1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [3., 1., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
        3., 1., 1., 1., 1., 1., 2., 1.],
       [2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1., 2., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
        1., 1., 1., 1., 